In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import cebra
from sklearn.preprocessing import StandardScaler

# Check if CUDA is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

In [ ]:
Contact_data = pd.read_feather("/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Skeleton_TNT/250106_FinalEventCutoffData_norm/contact_data/250106_Pooled_contact_data.feather")

In [ ]:
Contact_data.head()

# Data preparation 

Here we'll start by creating a key column, that should be a unique identifier for each contact in the dataset (basically one number per interaction between fly and contact_index)

Then we remove anything that is not tracking data. We also remove the ball centre to only keep the preprocessed tracking data. 

Finally, we save to h5 file and load it to cebra

In [ ]:
for col in Contact_data.columns:
    print(col)
    

In [ ]:
# Make a trial column that is an index for each unique combination of fly and contact_index

Contact_data['trial'] = Contact_data.groupby(['fly', 'contact_index']).ngroup()



In [ ]:
# For each trial, shift the time column so that the first time is 0

Contact_data['time_shifted'] = Contact_data.groupby('trial')['time'].transform(lambda x: x - x.min())

In [ ]:
Contact_data.columns

In [ ]:
# Prepare a list of columns to keep

columns_to_keep = ['frame', 'time', 'x_Head', 'y_Head', 'x_Thorax',
       'y_Thorax', 'x_Abdomen', 'y_Abdomen', 'x_Rfront', 'y_Rfront',
       'x_Lfront', 'y_Lfront', 'x_Rmid', 'y_Rmid', 'x_Lmid', 'y_Lmid',
       'x_Rhind', 'y_Rhind', 'x_Lhind', 'y_Lhind', 'x_Rwing', 'y_Rwing',
       'x_Lwing', 'y_Lwing', 'contact_index',
       'euclidean_distance', 'x_centre_preprocessed', 'y_centre_preprocessed',
       'fly', 'flypath', 'experiment', 'Nickname', 'Brain region', 'Date',
       'Genotype', 'Period', 'FeedingState', 'Orientation', 'Light',
       'Crossing', 'trial', 'time_shifted']

In [ ]:
Contact_data.head()

In [ ]:
# Features (coordinates)
feature_columns = ['x_Head', 'y_Head', 'x_Thorax', 'y_Thorax', 'x_Abdomen', 'y_Abdomen', 
                   'x_Rfront', 'y_Rfront', 'x_Lfront', 'y_Lfront', 'x_Rmid', 'y_Rmid', 
                   'x_Lmid', 'y_Lmid', 'x_Rhind', 'y_Rhind', 'x_Lhind', 'y_Lhind', 
                   'x_Rwing', 'y_Rwing', 'x_Lwing', 'y_Lwing', 'x_centre_preprocessed', 
                   'y_centre_preprocessed']

# Time dimension
time_column = 'time_shifted'

# Metadata (you can choose relevant columns for auxiliary variables)
metadata_columns = ['frame','fly', 'flypath', 'experiment', 'Nickname', 'Brain region', 'Date',
       'Genotype', 'Period', 'FeedingState', 'Orientation', 'Light',
       'Crossing', 'euclidean_distance', ]

X = Contact_data[feature_columns].values
T = Contact_data[time_column].values
metadata = Contact_data[metadata_columns]

In [ ]:
# 4. Prepare data lists for MultiCEBRA
X_list = []
T_list = []
auxiliary_list = []

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [ ]:
# 6. Iterate through trials
for trial in Contact_data['trial'].unique():
    trial_data = Contact_data[Contact_data['trial'] == trial]
    
    # Features
    X_trial = scaler.fit_transform(trial_data[feature_columns].values)
    X_list.append(X_trial)
    
    # Time
    T_trial = trial_data[time_column].values
    T_list.append(T_trial)
    
    # Auxiliary data (including time)
    auxiliary_trial = pd.get_dummies(trial_data[metadata_columns], columns=['fly', 'flypath', 'experiment', 'Nickname', 'Brain region', 'Date', 'Genotype', 'Period', 'FeedingState', 'Orientation', 'Light', 'Crossing'])
    auxiliary_trial = np.column_stack((T_trial, auxiliary_trial.values))
    auxiliary_list.append(auxiliary_trial)

In [ ]:
# Set up CEBRA model
model = cebra.CEBRA(
    model_architecture='offset10-model',
    batch_size=512,
    learning_rate=3e-4,
    temperature=1,
    output_dimension=3,  # You can adjust this
    max_iterations=10000,
    device=device,
    conditional='time_delta'
)

In [ ]:
model.fit(X_list, auxiliary=auxiliary_list)

In [ ]:
embeddings = model.transform(X_list)

In [ ]:
# Save it as h5 

Contact_data.to_hdf("/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Skeleton_TNT/250106_FinalEventCutoffData_norm/contact_data/250106_Pooled_contact_data.h5", key="df", mode="w", format="table")

In [ ]:

cebra_df = cebra.load_data("/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Skeleton_TNT/250106_FinalEventCutoffData_norm/contact_data/250106_Pooled_contact_data.h5")